# Imports

In [1]:
import requests
from sqlalchemy import create_engine, Table, Column, Float, String, MetaData, DateTime, insert, select, TIMESTAMP, text
import pandas as pd
import pickle
import time
from os import getenv
%load_ext dotenv
%dotenv

# Modell laden

In [2]:
# Pfad zur gespeicherten Modell-Datei
model_filename = 'karl_marx_arima_312_model.pkl'

# Laden des Modells
with open(model_filename, 'rb') as file:
    loaded_model = pickle.load(file)

# Verbindung zu remote postgresql

In [3]:
username = getenv('DB_USER')
password = getenv('DB_PASSWORD')
host = getenv('DB_HOST')
port = getenv('DB_PORT')
database = getenv('DB_NAME')

#remote postgreSQL Server
engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')

In [4]:
def upsert_prediction(timestamp, sensor_id, box_id, value):
    sql = text("""
        INSERT INTO predictions (TimeStamp, Sensor_ID, box_id, value)
        VALUES (:timestamp, :sensor_id, :box_id, :value)
        ON CONFLICT (TimeStamp, Sensor_ID, box_id) DO UPDATE SET
        value = EXCLUDED.value;
    """)
    with engine.connect() as conn:
        #  Parameter als Dictionary zu übergeben
        conn.execute(sql, {'timestamp': timestamp, 'sensor_id': sensor_id, 'box_id': box_id, 'value': value})
        conn.commit() 

In [5]:
def save_forecast_to_db(forecast, sensor_id, box_id):
    for timestamp, value in forecast.items():
        upsert_prediction(timestamp, sensor_id, box_id, value)

In [6]:
def fetch_newest_data(box_id, sensor_id, last_date):
    url = f"https://api.opensensemap.org/boxes/{box_id}/data/{sensor_id}"
    params = {
        'from-date': (last_date).isoformat('T')+'Z',
        'to-date': pd.Timestamp.now().isoformat('T')+'Z',
        'format': 'json'
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error: {response.status_code}")
        return []

# Box ID und Sensor_ID
box_id = "5984c712e3b1fa0010691509"
sensor_id = "5984c712e3b1fa001069150d"

#Zeitstempel der ersten Vorhersage(steps=2 weil sonst die stunden verloren gehen)
predictions = loaded_model.forecast(steps=2)
# Ziehen des letzten Indexes
last_entry = predictions.index[0]

print(f"from-date: ({last_entry.isoformat('T')+'Z'})")
print(f"to_date: ({pd.Timestamp.now().isoformat('T')+'Z'})")
data = fetch_newest_data(box_id, sensor_id, last_entry)
df = pd.DataFrame(data)


from-date: (2024-06-29T11:00:00Z)
to_date: (2024-06-29T19:15:05.162381Z)


In [ ]:
while True:
    #Model extenden bevor wir vorhersagen machen
    predictions = loaded_model.forecast(steps=2)
    # Ziehen des letzten Indexes
    last_entry = predictions.index[0]
    data = fetch_newest_data(box_id, sensor_id, last_entry)
    df = pd.DataFrame(data)
    if not df.empty:
        # Konvertierung des Timestamps in ein Pandas Datetime-Format
        df['createdAt'] = pd.to_datetime(df['createdAt'])
        df['value'] = pd.to_numeric(df['value'])
        # Setzen von 'timestamp' als Index der DataFrame
        df.set_index('createdAt', inplace=True)
        resampled_df_karl_marx_str_pm10 = df['value'].resample('h').mean()
        resampled_df_karl_marx_str_pm10.index.freq = 'h'
        # ZeitZone entfernen, damit das Modell es annehmen kann.
        resampled_df_karl_marx_str_pm10.index = resampled_df_karl_marx_str_pm10.index.tz_localize(None)
        # Modell erweitern
        model_extended = loaded_model.extend(resampled_df_karl_marx_str_pm10)
    
    # Vorhersagen für die nächsten vier Stunden machen und speichern
    forecast = model_extended.forecast(steps=24)
    
    box_id = "5984c712e3b1fa0010691509"
    sensor_id = '5984c712e3b1fa001069150d'  
    save_forecast_to_db(forecast, sensor_id, box_id)
    time.sleep(3600)  # In einer Stunde nächste Vorhersage

In [ ]:
model_filename = 'karl_marx_arima_312_model.pkl'
with open(model_filename, 'wb') as file:
    pickle.dump(model_extended, file)

model_filename